In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [2]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "2f40cbf66bmshb1de59ee1069bbcp1450a6jsnfa02a69d3f10"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

      flights = pd.DataFrame(list_for_df)
    return flights

In [3]:
icaos = ['EDDB', 'EGLL']
flights_df = tomorrows_flight_arrivals(icaos)

In [4]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   arrival_icao          168 non-null    object
 1   arrival_time_local    168 non-null    object
 2   arrival_terminal      168 non-null    object
 3   departure_city        168 non-null    object
 4   departure_icao        163 non-null    object
 5   departure_time_local  163 non-null    object
 6   airline               168 non-null    object
 7   flight_number         168 non-null    object
 8   data_retrieved_on     168 non-null    object
dtypes: object(9)
memory usage: 11.9+ KB


In [5]:
flights_df.head()

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2023-03-22 06:50+01:00,1,Doha,OTHH,2023-03-22 02:25+03:00,Qatar Airways,QR 79,2023-03-21
1,EDDB,2023-03-22 07:35+01:00,1,Duesseldorf,EDDL,2023-03-22 06:20+01:00,Eurowings,EW 9048,2023-03-21
2,EDDB,2023-03-22 07:35+01:00,0,Antalya,LTAI,2023-03-22 06:05+03:00,Pegasus,PC 5007,2023-03-21
3,EDDB,2023-03-22 07:40+01:00,1,Singapore,WSSS,2023-03-22 01:10+08:00,Scoot,TR 734,2023-03-21
4,EDDB,2023-03-22 07:50+01:00,1,Stuttgart,EDDS,2023-03-22 06:30+01:00,Eurowings,EW 2002,2023-03-21


In [6]:
flights_df['arrival_terminal'] = pd.to_numeric(flights_df['arrival_terminal'])

In [7]:
flights_df['data_retrieved_on'] = pd.to_datetime(flights_df['data_retrieved_on'])

In [8]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   arrival_icao          168 non-null    object        
 1   arrival_time_local    168 non-null    object        
 2   arrival_terminal      168 non-null    int64         
 3   departure_city        168 non-null    object        
 4   departure_icao        163 non-null    object        
 5   departure_time_local  163 non-null    object        
 6   airline               168 non-null    object        
 7   flight_number         168 non-null    object        
 8   data_retrieved_on     168 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 11.9+ KB


In [9]:
flights_df

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2023-03-22 06:50+01:00,1,Doha,OTHH,2023-03-22 02:25+03:00,Qatar Airways,QR 79,2023-03-21
1,EDDB,2023-03-22 07:35+01:00,1,Duesseldorf,EDDL,2023-03-22 06:20+01:00,Eurowings,EW 9048,2023-03-21
2,EDDB,2023-03-22 07:35+01:00,0,Antalya,LTAI,2023-03-22 06:05+03:00,Pegasus,PC 5007,2023-03-21
3,EDDB,2023-03-22 07:40+01:00,1,Singapore,WSSS,2023-03-22 01:10+08:00,Scoot,TR 734,2023-03-21
4,EDDB,2023-03-22 07:50+01:00,1,Stuttgart,EDDS,2023-03-22 06:30+01:00,Eurowings,EW 2002,2023-03-21
...,...,...,...,...,...,...,...,...,...
163,EDDB,2023-03-22 22:40+01:00,1,Aqaba,OJAQ,2023-03-22 19:50+03:00,easyJet,U2 5892,2023-03-21
164,EDDB,2023-03-22 22:45+01:00,1,Madrid,LEMD,2023-03-22 19:45+01:00,Iberia,IB 3674,2023-03-21
165,EDDB,2023-03-22 22:45+01:00,1,Tel Aviv Yafo,LLBG,2023-03-22 19:10+02:00,easyJet,U2 5206,2023-03-21
166,EDDB,2023-03-22 22:45+01:00,1,Funchal,LPMA,2023-03-22 17:10+00:00,easyJet,U2 5582,2023-03-21
